In [ ]:
#check graphics card
!nvidia-smi

Wed Jul 10 14:50:10 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#dependencies
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install wheel
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy
!pip install PyMuPDF
!pip install flask
!pip install flask-ngrok
!pip install pyngrok
!ngrok authtoken 2ixekPIkZLAlcygmMFr0lmvLaxF_7NYiGjjr6aVDSpUPGrXZY

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 24.1 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.1.78-cp310-cp310-linux_x86_64.whl size=294848 sha256=3ceccd30c5f3fd8bf7fabee0779cdd4541271409a6aa7e4f0374989631d7e135
  Stored in directory: /root/.cache/pip/wheels/61/f9/20/9ca660a9d3f2a47e44217059409478865948b5c8a1cba70030
Successfully built llama-cpp-python
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
INFO[07-11|07:58:35] no configuration paths supplied 
DBUG[07-11|07:58:35] ngrok config file at legacy location does not exist legacy_path=/root/.conf

In [ ]:
from flask import Flask, render_template, request
import fitz
from huggingface_hub import hf_hub_download
from llama_cpp import Llama
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

#Set up white paper text for LlaMa prompt
with fitz.open("sample.pdf", filetype='pdf') as doc: #change the name of the pdf file later
    text = ""
    for page in doc:
        text += page.get_text()

model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_ctx=2048,
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

#function to process context and user question with LlaMa
def model_output(question):
    template =f"""
Your purpose is to answer questions about AI, its various implications, factors that must be considered while implementing it, and the responsibilities of corporations and governments when developing and implementing AI.
Follow exactly these 3 steps:
1. Read the context below and aggregrate this data
Context :
{text}
2. Answer the question using only the above context, use NOTHING ELSE
Remember, you are an honest, helpful, and truthful assistant. Never make up answers on your own. If you don't have any context and are unsure of the answer, or if the question asked by the user is out of context, reply with a witty answer and tell them to ask questions within the context.

User Question: {question}

Response:
"""
    response = lcpp_llm(prompt=template, max_tokens=2048, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)
    output = response["choices"][0]["text"]
    return output.replace(f"""
Your purpose is to answer questions about AI, its various implications, factors that must be considered while implementing it, and the responsibilities of corporations and governments when developing and implementing AI.
Follow exactly these 3 steps:
1. Read the context below and aggregrate this data
Context :
{text}
2. Answer the question using only the above context, use NOTHING ELSE
Remember, you are an honest, helpful, and truthful assistant. Never make up answers on your own. If you don't have any context and are unsure of the answer, or if the question asked by the user is out of context, reply with a witty answer and tell them to ask questions within the context.
""", '')
#frontend flask app
app = Flask(__name__, template_folder='/content/templates')
run_with_ngrok(app)

@app.route("/", methods=['GET', 'POST'])
def welcome_page():
    return render_template("Webpage.html")

@app.route("/modelResponse", methods=['GET', 'POST'])
def question_replace():
    user_question = str(request.form["user_question"])
    output = model_output(user_question)
    return render_template("Webpage.html", model_response=output)

ngrok_tunnel = ngrok.connect(5000)
print('Public URL:', ngrok_tunnel.public_url)
if __name__== '__main__':
  app.run()

AVX = 1 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | VSX = 0 | 


Public URL: https://33db-35-240-167-218.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://33db-35-240-167-218.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 08:02:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 08:02:39] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 08:07:47] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Jul/2024 08:09:42] "GET / HTTP/1.1" 200 -
